In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import csv
import json

In [4]:
user_train_path = '../../train_valid_test/user_train.pkl'
user_valid_path = '../../train_valid_test/user_valid.pkl'
user_test_path = '../../train_valid_test/user_test.pkl'

In [5]:
with open(user_train_path, 'rb') as f:
    user_train_dict = pickle.load(f)
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)

In [6]:
user_id = 1
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])

[[1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907]]
[1566]
[48]


In [7]:
len(user_train_dict[user_id][0])

36

In [11]:
movie_summary_df = pd.read_csv('../../movie_summary_25_words.csv')
movie_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [12]:
user_df = pd.read_csv('../../users.csv')
user_df.head()

,user_id,user_profile
0,1,female k-12 student under 18 years of age
1,2,male self-employed above 56 years of age
2,3,male scientist between 25 to 34 years of age
3,4,male executive/managerial between 45 to 49 yea...
4,5,male writer between 25 to 34 years of age


In [13]:
ratings_df = pd.read_csv('../../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


### Creating the SASRec CTR Data

In [ ]:
sasrec_train_dict = {"user": [],"history_item_id": [],"history_rating": [],"item_id": [],"rating": []}
sasrec_valid_dict = {"user": [],"history_item_id": [],"history_rating": [],"item_id": [],"rating": []}
sasrec_test_dict = {"user": [],"history_item_id": [],"history_rating": [],"item_id": [],"rating": []}
user_list = list(user_train_dict.keys())

cnt =  0
for user in user_list:
    # cnt += 1
    # print(f"User: {user}")
    
    # sasrec_valid_dict['user'].append(user)
    # sasrec_test_dict['user'].append(user)

    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(f"user_rating_dict: {user_rating_dict}")

    train_valid_test_movies = user_train_dict[user][0][-21:]
    train_valid_test_movies.append(user_valid_dict[user][0])
    train_valid_test_movies.append(user_test_dict[user][0])
    # print(f"All train_valid_test_movies: {train_valid_test_movies}")

    train_valid_test_ratings = []
    for movie in train_valid_test_movies:
        movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
        if movie_rating >= 3:
            train_valid_test_ratings.append(1)
        else:
            train_valid_test_ratings.append(0)
    # print(f"All train_valid_test_ratings: {train_valid_test_ratings}")

    ### SASRec Train Data
    sasrec_train_dict['user'].append(user)
    sasrec_train_dict['history_item_id'].append(train_valid_test_movies[-23:-3])
    sasrec_train_dict['history_rating'].append(train_valid_test_ratings[-23:-3])
    sasrec_train_dict['item_id'].append(train_valid_test_movies[-3])
    sasrec_train_dict['rating'].append(train_valid_test_ratings[-3])

    ### SASRec Valid Data
    sasrec_valid_dict['user'].append(user)
    sasrec_valid_dict['history_item_id'].append(train_valid_test_movies[-22:-2])
    sasrec_valid_dict['history_rating'].append(train_valid_test_ratings[-22:-2])
    sasrec_valid_dict['item_id'].append(train_valid_test_movies[-2])
    sasrec_valid_dict['rating'].append(train_valid_test_ratings[-2])

    ### SASRec Test Data
    sasrec_test_dict['user'].append(user)
    sasrec_test_dict['history_item_id'].append(train_valid_test_movies[-21:-1])
    sasrec_test_dict['history_rating'].append(train_valid_test_ratings[-21:-1])
    sasrec_test_dict['item_id'].append(train_valid_test_movies[-1])
    sasrec_test_dict['rating'].append(train_valid_test_ratings[-1])

    # if cnt == 10:
    #     break

In [38]:
print(f"""sasrec_train_dict: {len(sasrec_train_dict['user'])}\n
    sasrec_valid_dict: {len(sasrec_valid_dict['user'])}\n
    sasrec_test_dict: {len(sasrec_test_dict['user'])}\n
    """)

sasrec_train_dict: 6040

    sasrec_valid_dict: 6040

    sasrec_test_dict: 6040

    


### Saving files

In [45]:
def save_to_csv(path, data_dict):
    with open(path, 'w+', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data_dict.keys())
        # Write header
        writer.writeheader()
        # Write data rows
        for i in range(len(data_dict['user'])):
            row = {key: data_dict[key][i] for key in data_dict.keys()}
            writer.writerow(row)
    print("CSV file created successfully.")

In [46]:
train_data_path = './final_data/movie/train.csv'
valid_data_path = './final_data/movie/valid.csv'
test_data_path = './final_data/movie/test.csv'

save_to_csv(train_data_path, sasrec_train_dict)
save_to_csv(valid_data_path, sasrec_valid_dict)
save_to_csv(test_data_path, sasrec_test_dict)

CSV file created successfully.
CSV file created successfully.
CSV file created successfully.


#### Getting unique number of items

In [47]:
ratings_df.movie_id.nunique()

3706